In [1]:
import modern_robotics as mr
import numpy as np
import numpy.matlib
numpy.set_printoptions(precision=16)

In [2]:
def vlogR(R):
    tr = (np.trace(R) - 1) / 2
    tr = min(max(tr, -1), 1)
    fai = np.arccos(tr)
    if fai == 0:
        w = np.array([0, 0, 0])
    else:
        w = fai / (2 * np.sin(fai)) * np.array([R[2, 1] - R[1, 2], R[0, 2] - R[2, 0], R[1, 0] - R[0, 1]])
    return w


def Registration(X, Y, Rn=None, tn=None, Ln=None):
    if X.shape[0] != 3 or Y.shape[0] != 3:
        raise ValueError('Each argument must have exactly three rows.')
    elif X.shape[1] != Y.shape[1]:
        raise ValueError('X and Y must have the same number of columns.')
    elif X.shape[1] < 3 or Y.shape[1] < 3:
        raise ValueError('X and Y must each have 3 or more columns.')
    Npoints = X.shape[1]
    Xbar = np.mean(X, axis=1)
    Ybar = np.mean(Y, axis=1)
    Xtilde = X - np.tile(Xbar, (Npoints, 1)).T
    Ytilde = Y - np.tile(Ybar, (Npoints, 1)).T
    
    H = np.dot(Xtilde, Ytilde.T)
    
    U, S, V = np.linalg.svd(H)

    idx = np.argsort(S)[::-1]
    S = S[idx]
    U = U[:,idx]
    V = V[idx,:].T


    R = np.dot(np.dot(V, np.diag([1, 1, np.linalg.det(np.dot(V, U.T))])), U.T)
    t = Ybar - np.dot(R, Xbar)
    if Rn is not None and tn is not None and Ln is not None:
        ep = np.linalg.norm(t - tn)
        eo = np.linalg.norm(vlogR(R/Rn))
        et = ep + eo * Ln
        return R, t, ep, eo, et
    else:
        return R, t


def rotationW(g, theta):

    if theta==0:
        w=np.array([[0],[0],[0]])
        
    else:
        w=1/(2*np.sin(theta)) * np.array([
                                        [g[2,1]-g[1,2]],
                                        [g[0,2]-g[2,0]],
                                        [g[1,0]-g[0,1]] ])

    return w
    

def rotationTheta( g ):
    tr=(np.trace(g[0:3, 0:3]) - 1) / 2 
    if tr>=1:
        tr=1

    elif tr<=-1:

        tr=-1

    theta=np.arccos(tr)

    return theta


def vlog(g):

    fai=rotationTheta(g)
    w=fai*rotationW(g,fai)
    w = w.reshape((3,))

    if fai == 0:

        p=g[0:3,3]

    else:

        A = np.eye(3)-0.5*mr.VecToso3(w)
        b = (2*np.sin(fai)-fai*(1+np.cos(fai))) / (2*fai*fai*np.sin(fai))

        B = np.dot(np.dot(b,mr.VecToso3(w)),mr.VecToso3(w)) 

        p = np.dot((A+B),g[0:3,3])
    
    kesi=np.r_[w,p]

    return kesi


def aMatrix( kesi, q ):

    w=kesi[0:3]
    v=kesi[3:6]
    bW=np.zeros((6,6))
    bW[0:3,0:3]=mr.VecToso3(w)
    bW[3:6,0:3]=mr.VecToso3(v)
    bW[3:6,3:6]=mr.VecToso3(w)
    n=np.linalg.norm(w)
    t=n*q
    if n==0:
        aM=q*np.eye(6)
    else:
        a0 = q * np.eye(6)

        a10 = ( ( 4 - t * np.sin(t) - 4 * np.cos(t) ) / 2 / n**2 )
        a11 = np.dot(a10,bW) 

        a20 = ( ( 4 * t - 5 * np.sin(t) + t * np.cos(t) ) / 2 / n**3 )
        a21 = np.dot(np.dot(a20,bW),bW)

        a30 = ((2-t*np.sin(t)-2*np.cos(t))/2/n**4)
        a31 = np.dot(np.dot(np.dot(a30,bW),bW),bW)

        a40 = ((2*t-3*np.sin(t)+t*np.cos(t))/2/n**5)
        a41 = np.dot(np.dot(np.dot(np.dot(a40,bW),bW),bW),bW)

        aM = a0 + a11 + a21 + a31 + a41

    return aM


def dexp( kesi, theta ):

    J = np.c_[aMatrix(kesi,theta), kesi]

    return J


def se3Translation( v,theta ):

    T=np.r_[np.c_[np.eye(3),v*theta],np.array([0,0,0,1]).reshape((1,4))]

    return T


def rotationMatrix( w,theta ):

    R= np.eye(3) + mr.VecToso3(w) * np.sin(theta) + np.dot(mr.VecToso3(w), mr.VecToso3(w)) * (1-np.cos(theta))

    return R


def se3Rotation( w,v,theta ):

    R=rotationMatrix(w,theta)
    p = np.dot(theta*np.eye(3)+(1-np.cos(theta))*mr.VecToso3(w)+(theta-np.sin(theta))* np.dot(mr.VecToso3(w),mr.VecToso3(w)),v)
    T=np.r_[np.c_[R,p.reshape((3,1))], np.array([0,0,0,1]).reshape(1,4)]

    return T
        

def se3Exp( kesi ):

    n1=np.linalg.norm(kesi[0:3])
    n2=np.linalg.norm(kesi[3:6])

    if n1==0 and n2==0:
        T=np.eye(4)

    elif n1==0:
        T=se3Translation(kesi[3:6]/n2,n2)

    else:
        T=se3Rotation(kesi[0:3]/n1,kesi[3:6]/n1,n1)

    return T


def fKin(xi,theta,n):
    
    T = np.eye(4)
    for i in range(0,n):
        T = np.dot(T,se3Exp(np.dot(xi[:,i],theta[i])))
    
    return T


def traditionalCalibration(xi0, vtheta, gm, M):

    xi = np.copy(xi0)

    M_home = np.eye(4)

    dq=np.zeros((4,1))
    N=np.size(vtheta,0)

    gn=np.zeros((4,4,N))
    dg=np.zeros((4,4,N))
    vLog=np.zeros((6,N))

    for i in range(0,N):

        gn[:,:,i]= mr.FKinBody(M_home, xi, vtheta[i,:])
        dg[:,:,i] = np.linalg.solve(gn[:,:,i].T, gm[:,:,i].T).T
        vLog[:,i]= vlog(dg[:,:,i])

    error=np.zeros((3,1))

    for i in range(0,N):

        error=error+ np.array([
            [np.linalg.norm(vLog[:,i])],
            [np.linalg.norm(vLog[3:6,i])],
            [np.linalg.norm(vLog[0:3,i])]])


    simJ = np.zeros((60,25))
    meanE = np.zeros((3,11))

    meanE[:,0]=(error/N).reshape((3,))

    convergence=np.zeros((10,2))

    for m in range(0,10):

        for i in range(0,N):

            gn[:,:,i]= mr.FKinBody(M_home, xi, vtheta[i,:])
            dg[:,:,i]= np.linalg.solve(gn[:,:,i].T,gm[:,:,i].T).T 
            vLog[:,i] = vlog(dg[:,:,i])

        simY=np.zeros((6*N,1))

        for i in range(0,N):

            simY[6*(i+1)-6 : 6 * (i+1), 0 ] = vLog[:,i]
        
        for k in range(0,N-1):

            simJ[0+6*k:6+6*k,0:7] = dexp(xi[:,0], vtheta[k,0])

            simJ[0+6*k:6+6*k,7:14]= np.dot( mr.Adjoint( se3Exp( np.dot(vtheta[k,0], xi[:,0])) ), dexp( xi[:,1], vtheta[k,1] ))

            tempSIMJ= np.dot(mr.Adjoint( np.dot(se3Exp( np.dot(vtheta[k,0], xi[:,0])), se3Exp( np.dot(vtheta[k,1], xi[:,1])))), dexp( xi[:,2], vtheta[k,2]))

            simJ[6*k:6+6*k,14:18] = tempSIMJ[:,3:7]

            simJ[0+6*k:6+6*k,18:25] = np.dot(mr.Adjoint( np.dot(np.dot(se3Exp( np.dot(vtheta[k,0], xi[:,0]) ), se3Exp( np.dot( vtheta[k,1], xi[:,1] ) )), se3Exp( np.dot( vtheta[k,2], xi[:,2] ) )) ),dexp(xi[:,3],vtheta[k,3]))

            dp = np.linalg.lstsq(simJ, simY, rcond=None)[0]


        xi[:,0] = (xi[:,0].reshape((6,1))+dp[0:6].reshape((6,1))).reshape((6,))
        xi[0:3,0] = xi[0:3,0] / np.linalg.norm(xi[0:3,0])
        xi[3:6,0] = xi[3:6,0] + np.dot( np.dot(xi[0:3,1].T, xi[3:6,0]) /  np.dot(xi[0:3,0].T, xi[0:3,0] ), xi[0:3,0])

        xi[:,1] = (xi[:,1].reshape((6,1)) + dp[7:13].reshape((6,1))).reshape((6,))
        xi[0:3,1] = xi[0:3,1]/np.linalg.norm(xi[0:3,1])
        xi[3:6,1] = xi[3:6,1] - np.dot( np.dot(xi[0:3,1].T, xi[3:6,1]) / np.dot(xi[0:3,1].T,xi[0:3,1]), xi[0:3,1])

        xi[3:6,2]=(xi[3:6,2].reshape((3,1)) + dp[14:17].reshape((3,1))).reshape((3,))
        xi[3:6,2]=xi[3:6,2] / np.linalg.norm(xi[3:6,2])

        xi[:,3]= (xi[:,3].reshape((6,1)) + dp[18:24].reshape((6,1))).reshape((6,))
        xi[0:3,3]=xi[0:3,3] / np.linalg.norm(xi[0:3,3])
        xi[3:6,3]= xi[3:6,3] - np.dot(np.dot(xi[0:3,3].T, xi[3:6,3]) / np.dot(xi[0:3,3].T,xi[0:3,3]),xi[0:3,3])

        vtheta[:,0] = vtheta[:,0]+dp[6]
        vtheta[:,1] = vtheta[:,1]+dp[13]
        vtheta[:,2] = vtheta[:,2]+dp[17]
        vtheta[:,3] = vtheta[:,3]+dp[24]

        dq = dq + np.array([dp[6], dp[13], dp[17], dp[24]])

        for i in range(0,N):
            gn[:,:,i] = mr.FKinBody(M_home, xi, vtheta[i,:])
            dg[:,:,i] = np.linalg.solve(gn[:,:,i].T,gm[:,:,i].T).T
            vLog[:,i] = vlog(dg[:,:,i])

        error=np.zeros((3,1))

        for i in range(0,N):
            
            error=error + np.r_[ np.r_[ np.linalg.norm(vLog[:,i]), np.linalg.norm(vLog[3:6,i]) ], np.linalg.norm(vLog[0:3,i]) ].reshape((3,1))

        meanE[:, m] = (error/N).reshape((3,))
        convergence[m,0]=np.linalg.norm(simY)
        convergence[m,1]=np.linalg.norm(dp)

    return xi, dq, meanE, convergence


In [5]:
w1 = np.array([0,0,1])
p1 = np.array([0,0,0])

w2 = np.array([0,0,1])
p2 = np.array([0.250,0,0])

w3 = np.array([0,0,0])
p3 = np.array([0,0,-1])

w4 = np.array([0,0,-1])
p4 = np.array([0.470,0,0])



s = 0.001
ax1 = mr.ScrewToAxis(p1,w1,0)
ax2 = mr.ScrewToAxis(p2,w2,0)
ax3 = np.r_[[0,0,0],p3]
ax4 = mr.ScrewToAxis(p4,w4,0)
xi0 = np.c_[ax1,ax2,ax3,ax4]


deq1=0
deq2=0.02
deq3=0.002
deq4=0.02

xi01= np.array([0.0199900035972015,0,0.999800179914059,0,0.0130330000000000*1000*s,0])
xi02=np.array([0,0.000399999968000004,0.999999920000010,-0.000300000000000000*1000*s,-0.253990000161600*1000*s,0.000101596000064640*1000*s])
xi03=np.array([0,0,0,0.0199999568791395,0.0195999577415567,-0.999607844797830])
xi04=np.array([0.0407700195329210,0.0391700187663605,-0.998400478333784,-0.0266829837144079*1000*s,0.504558015646471*1000*s,0.0187056011887379*1000*s])
xi00=np.c_[xi01,xi02,xi03,xi04]
N=10 

vtheta=np.c_[np.random.rand(N,1)*2*np.pi,np.random.rand(N,1)*2*np.pi,np.random.rand(N,1)*1000*s,np.random.rand(N,1)*2*np.pi]
P00=np.array([-100,-100,-100])*s
P01=np.array([100,0,0])*s
P02=np.array([0,100,0])*s
P03=np.array([0,0,100])*s
PX=np.c_[P01,P02,P03]
gm=np.zeros((4,4,N))
gn=np.zeros((4,4,N))
Pa1=np.zeros((4,N))
Pa2=np.zeros((4,N))
Pa3=np.zeros((4,N))
M_home = np.eye(4)


# vtheta = np.array([
#     [0.3044,    2.4914,    0.8905,    4.5410],
#     [4.1966,    0.3870,    0.7990,    0.6934],
#     [3.7917,    4.9020,    0.7343,    0.7382],
#     [3.3056,    2.1211,    0.0513,    4.0257],
#     [4.5849,    3.8193,    0.0729,    2.0660],
#     [4.4438,    4.6574,    0.0885,    4.1080],
#     [4.9095,    0.6586,    0.7984,    4.7069],
#     [1.8094,    0.8035,    0.9430,    3.6643],
#     [4.3513,    3.4529,    0.6837,    4.6498],
#     [3.4977,    3.0488,    0.1321,    1.4755]
# ])

for i in range(0,10):
    
    thetalist = np.array([ vtheta[i,0] + deq1, vtheta[i,1] + deq2, vtheta[i,2] + deq3, vtheta[i,3] + deq4 ])

    Pa1[:,i] = np.dot(mr.FKinBody(M_home, xi00, np.array([ vtheta[i,0] + deq1, vtheta[i,1] + deq2, vtheta[i,2] + deq3, vtheta[i,3] + deq4 ])), np.r_[P01,1].reshape((4,1)) ).reshape((4,)) + ((np.random.rand(4,1)*0.2-0.1) * s).reshape((4,))
    Pa2[:,i] = np.dot(mr.FKinBody(M_home, xi00, np.array([ vtheta[i,0] + deq1, vtheta[i,1] + deq2, vtheta[i,2] + deq3, vtheta[i,3] + deq4 ])), np.r_[P02,1].reshape((4,1)) ).reshape((4,)) + ((np.random.rand(4,1)*0.2-0.1) * s).reshape((4,))
    Pa3[:,i] = np.dot(mr.FKinBody(M_home, xi00, np.array([ vtheta[i,0] + deq1, vtheta[i,1] + deq2, vtheta[i,2] + deq3, vtheta[i,3] + deq4 ])), np.r_[P03,1].reshape((4,1)) ).reshape((4,)) + ((np.random.rand(4,1)*0.2-0.1) * s).reshape((4,))
    PY = np.c_[Pa1[0:3,i],Pa2[0:3,i],Pa3[0:3,i]]

    R,t = Registration(PX, PY)

    gm[0:3,0:3,i] = R
    gm[0:3,3,i] = t
    gn[:,:,i] = mr.FKinBody(M_home, xi00, thetalist)

# calibration
xiTrad, dqTrad, meanETrad, convergenceTrad = traditionalCalibration(xi0, vtheta, gm, 10)
Nt=50
error_before=np.zeros((Nt,1))
error_afterMinimal=np.zeros((Nt,1))
error_afterTraditional=np.zeros((Nt,1))
test_joint_config = np.c_[np.random.rand(Nt,1) * 2 * np.pi, np.random.rand(Nt,1) * 2 * np.pi, np.random.rand(Nt,1) * 1000 * s, np.random.rand(Nt,1) * 2 * np.pi ]


for i in range(0,Nt):
    
    error_before[i] = np.linalg.norm( np.dot( mr.FKinBody(M_home, np.round(xi0,4), test_joint_config[i,:]) - mr.FKinBody(M_home, np.round(xi00,4), test_joint_config[i,:] + np.array([deq1,deq2,deq3,deq4]) ), np.r_[P00,1])) / s
    
    error_afterTraditional[i] = np.linalg.norm( np.dot( mr.FKinBody(M_home, np.round(xiTrad,4), (test_joint_config[i,:] + dqTrad.T).reshape( (4,) ) ) - mr.FKinBody(M_home, np.round(xi00,4), test_joint_config[i,:] + np.array([deq1,deq2,deq3,deq4]) ), np.r_[P00,1]) ) / s


print("Mean error = {}".format(np.mean(error_before)))
print("Mean error calibrated = {}".format(np.mean(error_afterTraditional)))

print("Max error before = {}".format(np.max(error_before)))
print("Max error calibrated = {}".format(np.max(error_afterTraditional)))

Mean error = 93.85021700632161
Mean error calibrated = 0.4739833312571232
Max error before = 181.33175703577464
Max error calibrated = 0.8938368733834198
